In [1]:
import import_ipynb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import decomposition
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [2]:
from Data_investigation import Load_and_Combine
from Data_investigation import Filter_Data
from Data_investigation import Drop_Cols
from Data_investigation import Split_Col_Types
from Data_investigation import Restruct_Data
from Data_investigation import encode_and_concat
from Data_investigation import Show_Unique_Values
from Data_investigation import Count_Unique

importing Jupyter notebook from Data_investigation.ipynb


In [3]:
Users=pd.read_csv('./ka_users.csv')

In [4]:
path1='./ka_devices.db'
path2='./ka_users.csv'
path3='./ka_actions.parquet'
columns_drop=['game_stats_tutorial_complete','index','device_os_s','device_mapped_s','geo_s','device_manufacturer_s','device_model_s','device_gpu_s','lang_s']

In [5]:
df_F=Load_and_Combine(path1,path2,path3)

In [6]:
df_F,s_nlls=Filter_Data(df_F)

In [7]:
#Use this to show where the Na's are. #It is also verified that the amount of Na's are small see 'Filter_Data()'
#s_nlls

In [8]:
df_F=Drop_Cols(df_F,columns_drop)

In [9]:
df_F.total_spend.describe()

count    1.751748e+06
mean     1.867426e+00
std      1.713886e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      4.500000e+02
Name: total_spend, dtype: float64

In [10]:
types,cat_types,cluster_types= Split_Col_Types(df_F)

### Distinct Categorical values

In [11]:
### Use this to show how many distinct categorical values are present as well as the count. Consider grouping using regex and nltk 
#Show_Unique_Values(df_F,cat_types)
#Count_Unique(df_F,cat_types)

In [12]:
### Therefore remove more categorical data 
cat_types=[i for i in types['object'] if i not in columns_drop]

In [13]:
cluster_types

['device_mem_i',
 'device_gmem_i',
 'device_mem_grouping_i',
 'device_gmem_grouping_i',
 'device_numcpu_i',
 'device_height_i',
 'logins',
 'game_stats_tutorial_complete_time',
 'game_stats_xp',
 'game_stats_xp1',
 'game_stats_xp2',
 'game_stats_xp3',
 'redeemer_actions',
 'scribe_actions',
 'total_spend']

In [14]:
df_F=Restruct_Data(df_F,cluster_types)

In [15]:
df_F.device_mem_i.min()

-0.3987861276732424

In [20]:
df_F.describe()

,uid_s,region_s,tz_i,local_time_join_hour,local_time_join_dow,total_spend,platform_s,device_mem_i,device_gmem_i,device_mem_grouping_i,...,device_numcpu_i,device_height_i,logins,game_stats_tutorial_complete_time,game_stats_xp,game_stats_xp1,game_stats_xp2,game_stats_xp3,redeemer_actions,scribe_actions
count,1.751672e+06,1.751672e+06,1.751672e+06,1.751672e+06,1.751672e+06,1.751672e+06,1.751672e+06,1.751672e+06,1.751672e+06,1.751672e+06,...,1.751672e+06,1.751672e+06,1.751672e+06,1.751672e+06,1751672,1751672,1751672,1751672,1751672,1751672
unique,2.162000e+03,8.000000e+00,1.070000e+02,8.000000e+00,1.100000e+01,1.690000e+02,1.940000e+02,2.794700e+04,2.225700e+04,5.591000e+03,...,7.835000e+03,3.745000e+03,9.660000e+02,7.900000e+01,1751672,5,31,24,7,2
top,-2.296856e-01,2.944855e-01,-2.260122e-01,2.944855e-01,8.445697e-01,6.647335e-01,-4.699299e-01,-3.251942e-01,-3.635366e-01,-7.173117e-01,...,-1.718724e-01,-5.791520e-01,-7.592304e-01,-1.089597e-01,0,asia,28800,20,7,android
freq,4.339900e+04,8.149090e+05,1.762810e+05,8.149090e+05,9.361470e+05,5.855500e+05,7.965470e+05,3.982000e+03,1.945580e+05,2.273800e+05,...,1.600638e+06,8.670300e+04,6.612400e+04,1.720981e+06,1,972686,708564,127442,458458,1322479


In [17]:
X,y=df_F.drop(['total_spend'],axis=1),df_F.total_spend

In [18]:
X=encode_and_concat(X,cat_types)


In [19]:
dmatrix = xgb.DMatrix(data=X, label=y)
params = { 'max_depth': 100,
           'learning_rate': 0.3,
           'colsample_bytree': 0.3,
            'alpha': 50,
         'subsample': 0.6}
cv_results = xgb.cv(dtrain=dmatrix, 
                   params=params,
                   metrics='rmse', 
                   as_pandas=True,
                    num_boost_round=100,
                 early_stopping_rounds=20,
                   nfold=5)


ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:uid_s, tz_i, local_time_join_hour, local_time_join_dow, device_mem_i, device_gmem_i, device_mem_grouping_i, device_gmem_grouping_i, device_numcpu_i, device_height_i, logins, game_stats_tutorial_complete_time, game_stats_xp, game_stats_xp1, game_stats_xp2, game_stats_xp3, redeemer_actions, scribe_actions

In [ ]:
cv_results.head()

In [ ]:
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
xg_reg = xgb.train(params=params, dtrain=dmatrix, num_boost_round=100)


In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

In [ ]:
y.describe()

In [ ]:
### time taken to complete game may be important, investigate
### Logins seems important, investigate
### We already know game_stats_tutorial_complete is important 

#Use these variables for visualization to build a better profile. Additionally, check XGBoost 

In [ ]:
## using scaled data because we might add l1 parameter#


In [ ]:
#### Using the variable importance, and PCA. We want to investigate specific variables 

In [ ]:
# Game_stat_titorial_Complete_Time, redeemer_actions,device_mem,logins
### create bins for DataFrame 
### Use coplot
### Summary 
### Finish 


In [ ]:
column_types=df_F.columns.to_series().groupby(df_F.dtypes).groups
types={i.name: v for i, v in column_types.items()}
types
